In [1]:
import pyodbc
import pandas as pd

# Set your credentials here
server = 'DESKTOP-6NBPH33'  # or your instance name
database = 'DW_Finance'  # or the actual DB name
username = 'rana'  # if using SQL Server Authentication
password = 'rana'

# Use this if you're using SQL Server Authentication
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

conn = pyodbc.connect(conn_str)




In [5]:
import pyodbc
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Étape 1 : Connexion à la base de données
try:
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=DESKTOP-6NBPH33;'
        'DATABASE=DW_Finance;'
        'UID=rana;'
        'PWD=rana;'
    )
    print("Connexion réussie à la base de données.")
except Exception as e:
    print("Connexion échouée :", e)
    exit()

# Étape 2 : Récupération des données
query = """
SELECT 
    f.Fk_Produit,
    dp.ProductID,
    dp.ProductName,
    f.Price,
    f.Amount,
    f.DiscountOffered,
    f.RecommendedProfitMargin,
    fs.TotalAssets,
    fs.TotalLiabilities,
    fs.Equity,
    fs.Budget,
    fs.ActualExpenses,
    fs.AmountDue,
    fs.AmountPaid,
    fs.BalanceDue
FROM dbo.Fact_Purchase f
JOIN dbo.Dim_Produit dp ON f.Fk_Produit = dp.Pk_Produit
JOIN dbo.Fact_Financial_Situation fs ON f.Fk_Invoices = fs.Fk_Invoices
"""

# Chargement des données
df = pd.read_sql(query, conn)
print("\nDonnées chargées avec succès.")

# Étape 3 : Préparation des données
X = df[['Price', 'DiscountOffered', 'RecommendedProfitMargin', 'TotalAssets', 'TotalLiabilities', 'Equity', 'Budget', 'ActualExpenses']]
y = df['Budget'] - df['ActualExpenses']  # Variable cible

# Séparation train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✨ Data Augmentation simple : ajout d'un léger bruit aux features d'entraînement
noise_factor = 0.01
X_train_augmented = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
y_train_augmented = y_train  # Pas besoin de bruit sur y

# Étape 4 : Entraînement du modèle
model = LinearRegression()
model.fit(X_train_augmented, y_train_augmented)

# Étape 5 : Prédiction
y_pred = model.predict(X_test)

# Étape 6 : Évaluation
mse = mean_squared_error(y_test, y_pred)
print(f"\nErreur quadratique moyenne (MSE) : {mse:.10f}")

# Étape 7 : Affichage des résultats

# On remet les index correctement
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

df_predictions = pd.DataFrame({
    'Produit': df.loc[X_test.index, 'ProductName'].values,
    'Profit réel': y_test.values,
    'Profit prédit': y_pred
})

# Tri par profit réel
df_predictions = df_predictions.sort_values(by='Profit réel', ascending=False)

# Affichage du meilleur produit prédit
best_predicted_product = df_predictions.loc[df_predictions['Profit prédit'].idxmax()]
print("\nProduit recommandé avec le profit prédit le plus élevé :")
print(f"Produit : {best_predicted_product['Produit']}")
print(f"Profit prédit : {best_predicted_product['Profit prédit']:.2f}")

# (Optionnel : voir les 5 meilleurs)
#print("\nTop 5 produits prédits :")
#print(df_predictions.head(5))


Connexion réussie à la base de données.

Données chargées avec succès.

Erreur quadratique moyenne (MSE) : 0.0000017533

Produit recommandé avec le profit prédit le plus élevé :
Produit : Carottes Groupe Pomona
Profit prédit : 42553.00


C:\Users\MSI\AppData\Local\Temp\ipykernel_11640\1989434897.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
